In [1]:
from tools.ground_removal import *
from tools.raycasting import *
import numpy as np
from sklearn.cluster import DBSCAN
from mayavi import mlab
from tools.visualization import *
from scipy.spatial import KDTree
%load_ext autoreload
%autoreload 2

In [2]:
pts, intens = get_synchronized_frames_without_ground(0,1)

In [82]:
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], intens, mode='point')
mlab.points3d(0,0,0, mode='cube', color=(1,0,0))
mlab.show()

create objects through segmentation

In [3]:
clustering = DBSCAN(eps=0.6, min_samples=10).fit(pts[:,:3])
print(f"created {clustering.labels_.max() + 1} num of clusters")

created 182 num of clusters


In [84]:
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], clustering.labels_, mode='point')
mlab.show()

do not consider objects that are larger or smaller than limit sizes

In [4]:
valid_clusters = eliminate_objects_by_size(pts, clustering.labels_)
valid_mask = np.array([False * clustering.labels_.shape[0]])
print(f"discarted {valid_clusters.shape[0]} num of objects")
for cluster in valid_clusters:
    valid_mask = valid_mask | (clustering.labels_ == cluster)
valid_mask = valid_mask.astype(bool)

skipping cluster 0 because of its length 0.6355552672043068
skipping cluster 1 because of its length 0.9349546432599816
skipping cluster 2 because of its length 0.4089584348212334
skipping cluster 3 because of its length 12.671650293863499
skipping cluster 6 because of its height 2.670708656773204
skipping cluster 7 because of its width 4.237215040991259
skipping cluster 8 because of its height 3.405047059322218
skipping cluster 9 because of its length 6.3273773241636455
skipping cluster 10 because of its length 0.5831928250455967
skipping cluster 11 because of its length 38.23876571715943
skipping cluster 12 because of its width 3.8078346279070274
skipping cluster 13 because of its height 4.584777116869226
skipping cluster 14 because of its width 3.1459655761156213
skipping cluster 16 because of its width 3.2817535377969946
skipping cluster 17 because of its height 2.5151554314884477
skipping cluster 18 because of its length 13.527738567732136
skipping cluster 19 because of its height

In [86]:
mlab.points3d(pts[:,0][valid_mask], pts[:,1][valid_mask], pts[:,2][valid_mask], clustering.labels_[valid_mask], mode='point')
mlab.show()

Now, look at future frame(maybe three frames into future), look around where median of object O was at time t and check, if there are any points from time t+2 (+2 so that the car can move enough) if no and we can see the place at time t+2 (raycast) then it is dynamic

In [5]:
pts2, intens2 = get_synchronized_frames_without_ground(2,1)
clustering2 = DBSCAN(eps=0.6, min_samples=10).fit(pts2[:,:3])
print(f"created {clustering2.labels_.max() + 1} num of clusters")

created 189 num of clusters


In [88]:
mlab.points3d(pts2[:,0], pts2[:,1], pts2[:,2], clustering2.labels_, mode='point')
mlab.show()

In [6]:
valid_clusters2 = eliminate_objects_by_size(pts2, clustering2.labels_)
valid_mask2 = np.array([False * clustering2.labels_.shape[0]])
print(f"discarted {valid_clusters2.shape[0]} num of objects")
for cluster in valid_clusters2:
    valid_mask2 = valid_mask2 | (clustering2.labels_ == cluster)
valid_mask2 = valid_mask2.astype(bool)

skipping cluster 0 because of its length 0.6626111581996774
skipping cluster 2 because of its length 13.609034317186588
skipping cluster 4 because of its length 0.9997074654587479
skipping cluster 6 because of its length 6.468440292292847
skipping cluster 7 because of its width 4.400332289926567
skipping cluster 9 because of its length 6.6316310547163475
skipping cluster 10 because of its length 31.63218710948322
skipping cluster 11 because of its width 4.4423475954048115
skipping cluster 12 because of its height 4.291843349418136
skipping cluster 13 because of its height 3.0830041475080665
skipping cluster 14 because of its height 3.114554582327329
skipping cluster 15 because of its height 3.0618677812706867
skipping cluster 17 because of its width 3.0645114939388307
skipping cluster 18 because of its height 2.277743772260018
skipping cluster 19 because of its length 0.9749385924275593
skipping cluster 22 because of its height 2.5671169538494008
skipping cluster 23 because of its leng

In [68]:
mlab.points3d(pts2[:,0][valid_mask2], pts2[:,1][valid_mask2], pts2[:,2][valid_mask2], clustering2.labels_[valid_mask2], mode='point')
mlab.show()

In [7]:
tree = KDTree(pts2[:,:3])
moving_clusters = []
for cluster in valid_clusters:
    cluster_mask = clustering.labels_ == cluster
    centroid = np.mean(pts[:,:3][cluster_mask], axis=0)
    points_close_to_centroid = tree.query_ball_point(centroid, 1)
    if len(points_close_to_centroid) == 0:
        moving_clusters.append(cluster)
        
moving_clusters

[25, 48, 140]

In [32]:
mask = clustering.labels_ == 96
mlab.points3d(pts[:,0][mask], pts[:,1][mask], pts[:,2][mask], color=(1,0,0), mode='point')
#mlab.points3d(pts[:,0][~mask], pts[:,1][~mask], pts[:,2][~mask], color=(1,0,0), mode='point')
mlab.points3d(pts2[:,0], pts2[:,1], pts2[:,2], clustering2.labels_, mode='point')
mlab.show()

In [8]:
true_dynamic_objects = []
figure = mlab.figure()
origin = get_synchronized_origin(2)
for moving_cluster in moving_clusters:
    mask = clustering.labels_ == moving_cluster
    centroid = pts[mask][:,:3].mean(axis=0)
    response = send_raycast(pts2, target=centroid,
                            origin=origin, radius=0.3, fig=figure)
    if response is None:
        true_dynamic_objects.append(moving_cluster)
print(true_dynamic_objects)
mlab.points3d(pts2[:,0], pts2[:,1], pts2[:,2], clustering2.labels_, mode='point')
mask = clustering.labels_ == 48
mlab.points3d(pts[:,0][mask], pts[:,1][mask], pts[:,2][mask], color=(1,0,0), mode='point')
mlab.show()

Number of checks for raycast is 139
Number of checks for raycast is 105
ray has reached the destination without hitting anything
Number of checks for raycast is 168
[48]


In [62]:
pose = POSES[2]
pose = np.vstack((pose, [0, 0, 0, 1]))
transform_mat(np.array([0,0,0]).reshape(1,-1),pose)

array([[ 2.05582672, -0.10740306,  0.05404533]])

In [78]:
get_synchronized_origin(2)

array([ 2.05582672, -0.10740306,  0.05404533])

In [48]:
figure = mlab.figure()
mask = clustering.labels_ == 15
centroid = pts[mask][:,:3].mean(axis=0)

idx = send_raycast(pts, centroid, np.argwhere(mask), origin=[0,0,0], radius=0.5, fig=figure)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], intens, mode='point',figure=figure)
mlab.show()
print(idx)

Number of checks for raycast is 105
target [-40.69826448 -15.01589046   0.97422345] origin [5, 0, 0]
ray has hit target in points [6952, 3770, 3767]
None


In [9]:
mlab.points3d(0,0,0, mode='cube')
mlab.show()

In [66]:
centroid

array([-5.61959701, 36.24678809,  0.36734702])

In [76]:
np.sum(np.argwhere(mask) == idx)

1